In [1]:
from gdc.cms.data_access import *
from gdc.utils import filter_data, is_greater_than
import seaborn as sns

In [2]:
df_merged_payments.describe()

,payments_2008,payments_2009,payments_2010
count,112754.000000,112754.000000,112754.000000
mean,4031.517197,4331.417688,2521.582826
std,9897.083551,8578.550353,5703.427942
min,-430.000000,-1320.000000,-1770.000000
25%,0.000000,140.000000,70.000000
50%,960.000000,1550.000000,880.000000
75%,3080.000000,4270.000000,2290.000000
max,170190.000000,154970.000000,116910.000000


In [3]:
df_merged_payments.corr()

,payments_2008,payments_2009,payments_2010
payments_2008,1.000000,0.343507,0.170055
payments_2009,0.343507,1.000000,0.200042
payments_2010,0.170055,0.200042,1.000000


In [4]:
filter_data(
    df_merged_payments,
    {'payments_2008': is_greater_than(5000)}).mean()

payments_2008    18531.035898
payments_2009    10679.495039
payments_2010     4826.262663
dtype: float64

In [5]:
def relevant_cols(kw, df=df_merged_covariates):
    return [c for c in df.columns if c.find(kw) > -1]

relevant_cols(CB.cc_cancer)

['SP_CNCR_2_2008', 'SP_CNCR_2_2009', 'SP_CNCR_2_2010']

In [6]:
df_merged_covariates[relevant_cols(CB.cc_cancer)].corr()

,SP_CNCR_2_2008,SP_CNCR_2_2009,SP_CNCR_2_2010
SP_CNCR_2_2008,1.000000,0.429541,0.317119
SP_CNCR_2_2009,0.429541,1.000000,0.289550
SP_CNCR_2_2010,0.317119,0.289550,1.000000


In [7]:
df_merged_covariates[relevant_cols(CB.cc_esrd_indicator)].corr()

,BENE_ESRD_IND_Y_2008,BENE_ESRD_IND_Y_2009,BENE_ESRD_IND_Y_2010
BENE_ESRD_IND_Y_2008,1.000000,0.227804,0.172674
BENE_ESRD_IND_Y_2009,0.227804,1.000000,0.189684
BENE_ESRD_IND_Y_2010,0.172674,0.189684,1.000000


In [8]:
df_merged_covariates[relevant_cols(CB.cc_diabetes)].corr()

,SP_DIABETES_2_2008,SP_DIABETES_2_2009,SP_DIABETES_2_2010
SP_DIABETES_2_2008,1.000000,0.533124,0.394265
SP_DIABETES_2_2009,0.533124,1.000000,0.400199
SP_DIABETES_2_2010,0.394265,0.400199,1.000000


In [9]:
relevant_cols('2008', df_merged_payments)

['payments_2008']

In [10]:
y = df_merged_payments[relevant_cols('2008', df_merged_payments)]
X = sm.add_constant(df_merged_covariates[relevant_cols('2008')])

In [11]:
model = sm.OLS(y, X, missing='drop')
res = model.fit()

In [12]:
res.resid

DESYNPUF_ID
57FC84F24745CFA6    -1638.462702
0B582944B8C71C58       53.831037
8272FB0B28E248CC    44818.028967
9249DE518539A0D1    -7464.193772
52C2A0117A675ADB      428.936513
                        ...     
4658E62CA16335E3      388.563478
D7BE91E25C6F02F4       53.831037
350BE146D5D17455    -6740.519716
F245C5C422BC61F6      379.471243
3CF456D138A9C938      200.188731
Length: 112754, dtype: float64

In [26]:
y.values[:,0]

array([ 3000.,     0., 59120., ...,  1900.,     0.,     0.],
      shape=(112754,))

In [13]:
res.predict()

array([ 4638.4627021 ,   -53.83103678, 14301.97103256, ...,
        8640.51971561,  -379.47124285,  -200.1887313 ], shape=(112754,))

In [20]:
y.values[:,0] - res.predict()

array([-1638.4627021 ,    53.83103678, 44818.02896744, ...,
       -6740.51971561,   379.47124285,   200.1887313 ], shape=(112754,))